In [1]:
import pickle as pkl
import numpy as np
from collections import OrderedDict
from matplotlib import pyplot as plt
from pymc3.stats import quantiles
import os
import pandas as pd
from pathlib import Path
import sys

In [6]:
sys.path.append('/p/project/covid19dynstat/nieters1/BSTIM-Covid19/src')
from shared_utils import *

In [7]:
start = 42

with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)

n_weeks = 3 # 3 week windows cut out!

countyByName = make_county_dict()

# plot_county_names = {"covid19": [county]}

start_day = pd.Timestamp('2020-01-28') + pd.Timedelta(days=start)
year = str(start_day)[:4]
month = str(start_day)[5:7]
day = str(start_day)[8:10]

day_folder_path = "../figures/{}_{}_{}".format(year, month, day)
Path(day_folder_path).mkdir(parents=True, exist_ok=True) # poss change to new variant.

In [8]:
prediction_region = "germany"
data = load_data_n_weeks(start, n_weeks, prediction_region, counties)

start_day = pd.Timestamp('2020-01-28') + pd.Timedelta(days=start)
i_start_day = 0
day_0 = start_day + pd.Timedelta(days=n_weeks*7+5)
day_m5 = day_0 - pd.Timedelta(days=5)
day_p5 = day_0 + pd.Timedelta(days=5)

_, target, _, _ = split_data(
    data,
    train_start=start_day,
    test_start=day_0,
    post_test=day_p5)

county_ids = target.columns
# county_id = countyByName[county]

# Load our prediction samples
res = load_predictions(start, n_weeks)
res_trend = load_trend_predictions(start, n_weeks)


In [9]:
n_days = (day_p5 - start_day).days

prediction_samples = np.reshape(res['y'], (res['y'].shape[0], -1, 412)) 
prediction_samples = prediction_samples[:,i_start_day:i_start_day+n_days,:]

prediction_samples_trend = np.reshape(res_trend['y'], (res_trend['y'].shape[0],  -1, 412))
prediction_samples_trend = prediction_samples_trend[:,i_start_day:i_start_day+n_days,:]

ext_index = pd.DatetimeIndex([d for d in target.index] + \
        [d for d in pd.date_range(target.index[-1]+timedelta(1),day_p5-timedelta(1))])

In [11]:
prediction_quantiles = quantiles(prediction_samples, (5, 25, 75, 95)) 
prediction_quantiles_trend = quantiles(prediction_samples_trend, (5, 25, 75, 95)) 

In [12]:
prediction_mean = pd.DataFrame(
    data=np.mean(
        prediction_samples,
        axis=0),
    index=ext_index,
    columns=target.columns)
prediction_q25 = pd.DataFrame(
    data=prediction_quantiles[25],
    index=ext_index,
    columns=target.columns)
prediction_q75 = pd.DataFrame(
    data=prediction_quantiles[75],
    index=ext_index,
    columns=target.columns)
prediction_q5 = pd.DataFrame(
    data=prediction_quantiles[5],
    index=ext_index,
    columns=target.columns)
prediction_q95 = pd.DataFrame(
    data=prediction_quantiles[95],
    index=ext_index,
    columns=target.columns)

prediction_mean_trend = pd.DataFrame(
    data=np.mean(
        prediction_samples_trend,
        axis=0),
    index=ext_index,
    columns=target.columns)
prediction_q25_trend = pd.DataFrame(
    data=prediction_quantiles_trend[25],
    index=ext_index,
    columns=target.columns)
prediction_q75_trend = pd.DataFrame(
    data=prediction_quantiles_trend[75],
    index=ext_index,
    columns=target.columns)
prediction_q5_trend = pd.DataFrame(
    data=prediction_quantiles_trend[5],
    index=ext_index,
    columns=target.columns)
prediction_q95_trend = pd.DataFrame(
    data=prediction_quantiles_trend[95],
    index=ext_index,
    columns=target.columns)

In [13]:
prediction_mean

,03159,09576,07334,06631,10046,01058,03459,05316,15089,04011,...,11003,11004,11006,11007,11008,11009,11010,11011,11012,11005
2020-03-10,1.919,1.058,1.670,6.434,2.709,2.107,2.601,1.814,1.760,7.845,...,4.258,6.770,4.858,3.881,2.606,2.513,1.422,1.442,1.996,1.013
2020-03-11,2.304,1.263,1.777,6.861,2.986,2.395,3.009,2.293,1.905,8.475,...,4.923,8.061,5.394,4.534,3.335,2.892,1.548,1.789,2.479,1.223
2020-03-12,2.737,1.439,1.857,7.061,3.335,2.818,3.997,2.952,1.994,10.424,...,6.173,9.473,6.621,6.078,4.402,3.832,2.077,2.361,3.131,1.544
2020-03-13,2.882,1.487,1.672,6.300,2.960,2.643,4.233,2.611,1.829,8.889,...,6.028,8.754,5.710,5.536,3.970,3.335,1.972,2.332,2.939,1.539
2020-03-14,2.366,1.281,1.338,5.032,2.359,2.129,3.563,2.201,1.365,6.431,...,4.379,6.340,4.096,4.375,3.155,2.667,1.473,1.624,2.279,1.125
2020-03-15,2.159,1.176,1.119,3.656,1.778,1.945,2.973,1.648,1.020,5.506,...,3.992,5.592,3.861,3.962,3.172,2.407,1.252,1.439,2.138,0.975
2020-03-16,3.969,1.985,1.809,6.126,3.277,3.456,5.773,2.365,1.821,9.420,...,7.494,10.816,6.824,7.702,6.063,4.303,2.392,2.842,3.953,2.086
2020-03-17,5.350,2.564,2.605,6.958,4.145,4.357,8.516,3.546,2.107,12.408,...,9.785,15.005,9.491,11.056,8.524,5.482,3.376,3.797,5.692,2.881
2020-03-18,6.495,3.313,3.137,7.848,4.538,5.213,10.644,5.284,2.314,13.637,...,13.685,19.625,11.646,13.973,12.249,6.820,4.121,5.029,7.976,3.818
2020-03-19,8.366,4.766,3.764,9.185,5.110,6.225,12.718,5.369,2.439,16.201,...,15.122,21.664,14.499,15.779,13.668,7.997,5.052,5.551,9.172,4.781
